# 참고

로컬 환경에서 작성된 코드 입니다.

데이터셋
- https://github.com/bab2min/corpus/tree/master/sentiment
- https://github.com/e9t/nsmc
- https://github.com/ko-nlp/Korpora

모델
- https://github.com/kiyoungkim1/LMkor
- https://github.com/SKTBrain/KoBERT
- https://github.com/monologg/KoBERT-Transformers

DOCS
- https://huggingface.co/docs/transformers/training
- https://wikidocs.net/44249
- https://arxiv.org/pdf/1810.04805.pdf (https://mino-park7.github.io/nlp/2018/12/12/bert-%EB%85%BC%EB%AC%B8%EC%A0%95%EB%A6%AC/)
- https://arxiv.org/pdf/2006.04884.pdf (https://chloelab.tistory.com/33)

# 라이브러리 로드

In [1]:
import re
import os

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from transformers import BertTokenizerFast, TFBertModel

import tensorflow as tf
import tensorflow_addons as tfa

from tqdm import tqdm
from IPython.display import clear_output 


ROOT_PATH = '../'

DATASET_PATH = ROOT_PATH + '/data'
CHECKPOINT_PATH = ROOT_PATH + '/saved_models'
MODEL_PATH = CHECKPOINT_PATH
CACHE_PATH = ROOT_PATH + '/cache'


def init_dir(path):
  if not os.path.exists(path):
    os.makedirs(path)


init_dir(DATASET_PATH)
init_dir(CHECKPOINT_PATH)
init_dir(MODEL_PATH)
init_dir(CACHE_PATH)

/root/workspace_github/KoSA/.venv/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2022-07-08 22:49:14.233903: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-07-08 22:49:14.233937: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-07-08 22:49:16.310474: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-07-08 22:49:16.310503: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)


# 데이터 로드

In [ ]:
!wget https://raw.githubusercontent.com/bab2min/corpus/master/sentiment/naver_shopping.txt -O corpus_naver_shopping.txt
!wget https://raw.githubusercontent.com/bab2min/corpus/master/sentiment/steam.txt -O corpus_steam.txt
!wget https://raw.githubusercontent.com/e9t/nsmc/master/ratings.txt -O corpus_naver_movie.txt

!mv corpus*.txt '{DATASET_PATH}'

clear_output()

In [2]:
df_1 = pd.read_table(DATASET_PATH + '/corpus_naver_shopping.txt', names=['label', 'content'])
df_1['label'] = df_1['label'].apply(lambda x: 1 if x > 3 else 0)

In [3]:
df_2 = pd.read_table(DATASET_PATH + '/corpus_steam.txt', names=['label', 'content'])

In [4]:
df_3 = pd.read_table(DATASET_PATH + '/corpus_naver_movie.txt', names=['id', 'content', 'label'])
df_3 = df_3[1:]
df_3 = df_3[['content', 'label']]

In [5]:
df = pd.concat([df_1, df_2, df_3])
df = df.drop_duplicates('content')
df = df.reset_index(drop=True)
df = df.astype({'label': np.uint8})
df = df.dropna()

del df_1
del df_2
del df_3

In [6]:
df.shape

(493987, 2)

In [7]:
X_train, X_test, y_train, y_test = train_test_split(df['content'], df['label'], test_size=0.3, random_state=0xC0FFEE)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((345790,), (148197,), (345790,), (148197,))

# 텍스트 데이터 전처리

In [8]:
tokenizer = BertTokenizerFast.from_pretrained('kykim/bert-kor-base', cache_dir=CACHE_PATH)

In [ ]:
# print(tokenizer.get_vocab())

In [9]:
print(tokenizer.tokenize('사전 학습된 토크나이저에 대한 테스트입니다.'))

['사전', '학습', '##된', '토크', '##나이', '##저', '##에', '대한', '테스트', '##입니다', '.']


In [10]:
print(tokenizer.encode('사전 학습된 토크나이저에 대한 테스트입니다.'))

[2, 17878, 16784, 8544, 30605, 20491, 8255, 8008, 14083, 15682, 13992, 2016, 3]


In [11]:
print(tokenizer.convert_ids_to_tokens(tokenizer.encode('사전 학습된 토크나이저에 대한 테스트입니다.')))

['[CLS]', '사전', '학습', '##된', '토크', '##나이', '##저', '##에', '대한', '테스트', '##입니다', '.', '[SEP]']


In [12]:
df['content'].str.len().sort_values(ascending=False)[:5]

373223    142
443320    140
40940     140
449034    140
422765    140
Name: content, dtype: int64

In [13]:
len(tokenizer.tokenize(df['content'][443320]))

70

In [14]:
tokenizer.encode_plus('사전 학습된 토크나이저에 대한 테스트입니다.', max_length=30, padding='max_length')

{'input_ids': [2, 17878, 16784, 8544, 30605, 20491, 8255, 8008, 14083, 15682, 13992, 2016, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}

In [15]:
SEQ_LEN = 128 # 최대 token 개수 이상의 값으로 임의로 설정

token_ids =[]
token_masks =[]
token_segments =[]

train_labels = []


for idx in tqdm(range(len(X_train))):
  
  train_sentence = X_train.iloc[idx]
    
  # 특수문자 제거
  cleaned_sentence = re.sub('[^\s0-9a-zA-Zㄱ-ㅎㅏ-ㅣ가-힣]', '', train_sentence)

  # Tokenizing / Tokens to sequence numbers / Padding
  encoded_dict = tokenizer.encode_plus(text=cleaned_sentence,
                                       padding='max_length', 
                                       truncation = True,
                                       max_length=SEQ_LEN)

  token_ids.append(encoded_dict['input_ids']) # tokens_tensor
  token_masks.append(encoded_dict['attention_mask']) # masks_tensor
  token_segments.append(encoded_dict['token_type_ids']) # segments_tensor

  train_labels.append(y_train.iloc[idx])


train_inputs = (np.array(token_ids), np.array(token_masks), np.array(token_segments))
train_labels = np.array(train_labels)

100%|██████████| 345790/345790 [00:51<00:00, 6739.91it/s]


In [16]:
train_inputs[0]

array([[    2, 22724, 13970, ...,     0,     0,     0],
       [    2,  7465, 14783, ...,     0,     0,     0],
       [    2,  2120,  8402, ...,     0,     0,     0],
       ...,
       [    2,  4886, 26227, ...,     0,     0,     0],
       [    2, 14600, 30245, ...,     0,     0,     0],
       [    2, 16476,  8120, ...,     0,     0,     0]])

In [17]:
print(train_inputs[0][1]) # tokens_tensor
print(train_inputs[1][1]) # masks_tensor 
print(train_inputs[2][1]) # segments_tensor
print(tokenizer.decode(train_inputs[0][1])) # Re-constructed sentence

[    2  7465 14783 14005  6169  8399 32020     3     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0]
[1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

# 모델링

In [2]:
def create_model(max_length):
  base_model = TFBertModel.from_pretrained('kykim/bert-kor-base', cache_dir=CACHE_PATH, from_pt=True)

  input_token_ids   = tf.keras.layers.Input((max_length,), dtype=tf.int32, name='input_token_ids')   # tokens_tensor
  input_masks       = tf.keras.layers.Input((max_length,), dtype=tf.int32, name='input_masks')       # masks_tensor
  input_segments    = tf.keras.layers.Input((max_length,), dtype=tf.int32, name='input_segments')    # segments_tensor  

  bert_outputs = base_model([input_token_ids, input_masks, input_segments]) 
  
  bert_outputs = bert_outputs[1] # ('pooler_output', <KerasTensor: shape=(None, 768) dtype=float32 (created by layer 'tf_bert_model')>)
  bert_outputs = tf.keras.layers.Dropout(0.2)(bert_outputs)
  final_output = tf.keras.layers.Dense(units=2,
                                       activation='softmax', 
                                       kernel_initializer=tf.keras.initializers.TruncatedNormal(stddev=0.02), 
                                       name='classifier')(bert_outputs)

  model = tf.keras.Model(inputs=[input_token_ids, input_masks, input_segments], 
                         outputs=final_output)
  
  model.compile(optimizer=tfa.optimizers.RectifiedAdam(learning_rate=1e-5, weight_decay=0.0025, warmup_proportion=0.05),
                loss=tf.keras.losses.SparseCategoricalCrossentropy(), 
                metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])
  
  return model

In [19]:
model = create_model(max_length=SEQ_LEN)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.LayerNorm.weight', 'bert.embeddings.position_ids', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.predictions.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the 

In [20]:
print(model.summary())

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_token_ids (InputLayer)   [(None, 128)]        0           []                               
                                                                                                  
 input_masks (InputLayer)       [(None, 128)]        0           []                               
                                                                                                  
 input_segments (InputLayer)    [(None, 128)]        0           []                               
                                                                                                  
 tf_bert_model (TFBertModel)    TFBaseModelOutputWi  118297344   ['input_token_ids[0][0]',        
                                thPoolingAndCrossAt               'input_masks[0][0]',        

# 모델 학습

In [21]:
# For custom models, we have to use "save_weights_only = True" (or we should implement a "get_config" method @ https://j.mp/3ltUibd) 
callback_checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath=CHECKPOINT_PATH + '/best_weights.h5',
                                                         monitor='val_sparse_categorical_accuracy',
                                                         save_best_only=True, 
                                                         save_weights_only = True, 
                                                         verbose=1)

# Early-stopping for preventing the overfitting
callback_earlystop = tf.keras.callbacks.EarlyStopping(monitor='val_sparse_categorical_accuracy', 
                                                      min_delta=0.0001, # the threshold that triggers the termination (acc should at least improve 0.0001)
                                                      patience=5) #  Number of epochs with no improvement after which training will be stopped

In [22]:
history = model.fit(train_inputs, train_labels, validation_split=0.2,
                    epochs=3, batch_size=16,
                    verbose=1,
                    callbacks=[callback_checkpoint, callback_earlystop])

Epoch 1/3


: 

: 

In [ ]:
import matplotlib.pyplot as plt 
import seaborn as sns

sns.set()

In [ ]:
acc = history.history['sparse_categorical_accuracy']
val_acc = history.history['val_sparse_categorical_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

fig = plt.figure(figsize=(20, 6))
graph1 = fig.add_subplot(1, 2, 1)
graph2 = fig.add_subplot(1, 2, 2)

ax1 = sns.lineplot(data=acc, label='acc', ax=graph1)
sns.lineplot(data=val_acc, label='val_acc', ax=graph1)
ax2 = sns.lineplot(data=loss, label='loss', ax=graph2)
sns.lineplot(data=val_loss, label='val_loss', ax=graph2)

ax1.set_xlabel('Epoch')
ax1.set_ylabel('Accuracy')
ax2.set_xlabel('Epoch')
ax2.set_ylabel('Loss')
plt.show()

# 모델 평가

위 train 데이터의 전처리와 동일하게 진행

In [ ]:
SEQ_LEN = 128

token_ids =[]
token_masks =[]
token_segments =[]

test_labels = []


for idx in tqdm(range(len(X_test))):
  
  test_sentence = X_test.iloc[idx]
    
  # 특수문자 제거
  cleaned_sentence = re.sub('[^\s0-9a-zA-Zㄱ-ㅎㅏ-ㅣ가-힣]', '', test_sentence)

  # Tokenizing / Tokens to sequence numbers / Padding
  encoded_dict = tokenizer.encode_plus(text=cleaned_sentence,
                                       padding='max_length', 
                                       truncation = True,
                                       max_length=SEQ_LEN)

  token_ids.append(encoded_dict['input_ids']) # tokens_tensor
  token_masks.append(encoded_dict['attention_mask']) # masks_tensor
  token_segments.append(encoded_dict['token_type_ids']) # segments_tensor

  test_labels.append(y_test.iloc[idx])

  
test_inputs = (np.array(token_ids), np.array(token_masks), np.array(token_segments))
test_labels = np.array(test_labels)

In [ ]:
model.load_weights(filepath=CHECKPOINT_PATH + '/best_weights.h5')

pred = model.predict(test_inputs)
pred = tf.argmax(pred, axis=1)

print(accuracy_score(pred, y_test))

# 모델 저장

In [4]:
import pickle
import dill

with open(MODEL_PATH + '/model_v1.pkl', 'wb') as f:
  pickle.dump(dill.dumps(create_model), f)

# 배포 코드

Server-Side

In [ ]:
from IPython.display import clear_output 

!pip install transformers
!pip install tensorflow_addons

clear_output()

In [5]:
from transformers import BertTokenizerFast, TFBertModel
import tensorflow as tf
import tensorflow_addons as tfa
import re
import numpy as np
import pickle
import dill

SEQ_LEN = 128
ROOT_PATH = '../'

CHECKPOINT_PATH = ROOT_PATH + '/saved_models'
MODEL_PATH = CHECKPOINT_PATH
CACHE_PATH = ROOT_PATH + '/cache'


def predict_sentiment(sentence, tokenizer, model):
  encoded_dict = tokenizer.encode_plus(text=re.sub('[^\s0-9a-zA-Zㄱ-ㅎㅏ-ㅣ가-힣]', '', sentence),
                                       padding='max_length', 
                                       truncation = True,
                                       max_length=SEQ_LEN)
  
  token_ids = np.array(encoded_dict['input_ids']).reshape(1, -1)
  token_masks = np.array(encoded_dict['attention_mask']).reshape(1, -1)
  token_segments = np.array(encoded_dict['token_type_ids']).reshape(1, -1)  

  return model.predict((token_ids, token_masks, token_segments))


# tokenizer
tokenizer = BertTokenizerFast.from_pretrained('kykim/bert-kor-base', cache_dir=CACHE_PATH)

# model
with open(MODEL_PATH + '/model_v1.pkl', 'rb') as f:
  create_model = dill.loads(pickle.load(f))

model = create_model(max_length=128)
model.load_weights(filepath=CHECKPOINT_PATH + '/best_weights.h5')

Downloading: 100%|██████████| 725/725 [00:00<00:00, 339kB/s]
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'bert.embeddings.position_ids', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initia

Client-Side

In [6]:
def test(sentence):
  prediction = predict_sentiment(sentence, tokenizer, model) # function -> API
  predicted_probability = np.round(np.max(prediction) * 100, 2)
  predicted_class = ['부정', '긍정'][np.argmax(prediction, axis=1)[0]]
  print('{}: {}% 확률로 {}입니다.'.format(sentence, predicted_probability, predicted_class))

In [7]:
texts = [
         'GOTY',
         'GOD GAME',
         'TRASH GAME',
         '쿠소겜',
         'ㅆㄹㄱ',
         '가껨',
]

for text in texts:
  test(text)

1/1 [==============================] - 2s 2s/step
GOTY: 90.35% 확률로 긍정입니다.
1/1 [==============================] - 0s 132ms/step
GOD GAME: 84.44% 확률로 긍정입니다.
1/1 [==============================] - 0s 134ms/step
TRASH GAME: 77.73% 확률로 부정입니다.
1/1 [==============================] - 0s 129ms/step
쿠소겜: 74.0% 확률로 부정입니다.
1/1 [==============================] - 0s 129ms/step
ㅆㄹㄱ: 70.72% 확률로 부정입니다.
1/1 [==============================] - 0s 127ms/step
가껨: 74.86% 확률로 긍정입니다.
